In [ ]:
from sage.crypto.sbox import *
from sage.crypto.sboxes import *
def butterfly(S1, S2, alpha):
    S1_inv = S1.inverse()
    results = []
    for xlMain in input:
        ls = []
        for xrMain in input:
            xl = xlMain
            xr = xrMain
            xl = xl + S1[xr] # phase 1
            xl = S1_inv[xl] + (alpha * xr) # phase 2
            temp = xl # swap
            xl = xr
            xr = temp
            xl = xl + (alpha * xr) # phase 3
            xl = S2[xl] + S2[xr] # phase 4
            xlStr = "{0:b}".format(xl.integer_representation()).zfill(3)
            xrStr = "{0:b}".format(xr.integer_representation()).zfill(3)
            ls.append(xlStr + xrStr)
        results.append(ls)

    lsresult = []
    for k in range(0,8):
        for j in range(0,8):
            output = int(results[k][j], 2)
            lsresult.append(output)
    return SBox(lsresult)

def double_butterfly(S1, S2, S3, S4, alpha, alpha2):
    results = []
    S1_inv = S1.inverse()
    S3_inv = S3.inverse()
    for xlMain in input:
        ls = []
        for xrMain in input:
            xl = xlMain
            xr = xrMain
            # First butterfly
            xl = xl + S1[xr] # phase 1
            xl = S1_inv[xl] + (alpha * xr) # phase 2
            temp = xl # swap
            xl = xr
            xr = temp
            xl = xl + (alpha * xr) # phase 3
            xl = S2[xl] + S2[xr] # phase 4
            # Second butterfly
            xr = xr + S3[xl] # phase 1
            xr = S3_inv[xr] + (alpha2 * xl) # phase 2
            temp = xr # swap
            xr = xl
            xl = temp
            xr = xr + (alpha2 * xl) # phase 3
            xr = S4[xr] + S4[xl] # phase 4

            xlStr = "{0:b}".format(xl.integer_representation()).zfill(3)
            xrStr = "{0:b}".format(xr.integer_representation()).zfill(3)
            ls.append(xlStr + xrStr)
        results.append(ls)

    lsresult = []
    for i in range(0,8):
        for j in range(0,8):
            output = int(results[i][j], 2)
            lsresult.append(output)         
    return SBox(lsresult)

def triple_butterfly(S1, S2, S3, S4, S5, S6, alpha):
    results = []
    S_inv1 = S1.inverse()
    S_inv3 = S3.inverse()
    S_inv5 = S5.inverse()
    for xlMain in input:
        ls = []
        for xrMain in input:
            xl = xlMain
            xr = xrMain
            # First butterfly
            xl = xl + S1[xr] # phase 1
            xl = S_inv1[xl] + (alpha * xr) # phase 2
            temp = xl # swap
            xl = xr
            xr = temp
            xl = xl + (alpha * xr) # phase 3
            xl = S2[xl] + S2[xr] # phase 4
            # Second butterfly
            xr = xr + S3[xl] # phase 1
            xr = S_inv3[xr] + (alpha * xl) # phase 2
            temp = xr # swap
            xr = xl
            xl = temp
            xr = xr + (alpha * xl) # phase 3
            xr = S4[xr] + S4[xl] # phase 4
            # Third butterfly
            xl = xl + S5[xr] # phase 1
            xl = S_inv5[xl] + (alpha * xr) # phase 2
            temp = xl # swap
            xl = xr
            xr = temp
            xl = xl + (alpha * xr) # phase 3
            xl = S6[xl] + S6[xr] # phase 4

            xlStr = "{0:b}".format(xl.integer_representation()).zfill(3)
            xrStr = "{0:b}".format(xr.integer_representation()).zfill(3)
            ls.append(xlStr + xrStr)
        results.append(ls)

    lsresult = []
    for i in range(0,8):
        for j in range(0,8):
            output = int(results[i][j], 2)
            lsresult.append(output)
    return SBox(lsresult)

# Replicate butterfly structure - 3-bits
F.<a> = GF(2**3, name = 'a')
L = F.list()
input = [L[0], L[7], L[1], L[3], L[2], L[6], L[4], L[5]]

power = [z**3 for z in input]
S = SBox(power)
S_inv = S.inverse()

for alpha in L:
    Sresult = butterfly(S, S, alpha)
    print(alpha, "Differential uniformity: ", Sresult.differential_uniformity(), "Permutation", Sresult.is_permutation())
    
# Replicate butterfly structure - 3-bit APN
F.<a> = GF(2**3, name = 'a', modulus = "primitive")
L = F.list()
input = [L[0], L[7], L[1], L[3], L[2], L[6], L[4], L[5]]

threebit= [('cipher', (0, 1, 3, 6, 7, 4, 5, 2)), ('SEA', (0, 5, 6, 7, 4, 3, 1, 2)), ('ctc2', (7, 6, 0, 4, 2, 5, 1, 3)), ('3way', (7, 2, 4, 5, 1, 6, 3, 0))]

for a in threebit:
    for b in threebit:
        for alpha in L:
            S1 = SBox(a[1])
            S2 = SBox(b[1])
            Sresult = butterfly(S1, S2, alpha)
            print(a[0], b[0], alpha, "Differential uniformity: ", Sresult.differential_uniformity(), "Permutation", Sresult.is_permutation())
            

# Double butterfly structure with power components - 3bits
F.<a> = GF(2**3, name = 'a')
L = F.list()
input = [L[0], L[7], L[1], L[3], L[2], L[6], L[4], L[5]]

power = [z**3 for z in input]
S = SBox(power)
S_inv = S.inverse()

for alpha in L:
    Sresult = double_butterfly(S, S, S, S, alpha, alpha)
    print(alpha, Sresult.differential_uniformity())

# Double butterfly structure using APN components - 3-bit
F.<a> = GF(2**3, name = 'a')
L = F.list()
input = [L[0], L[7], L[1], L[3], L[2], L[6], L[4], L[5]]
threebit= [('cipher', (0, 1, 3, 6, 7, 4, 5, 2)), ('SEA', (0, 5, 6, 7, 4, 3, 1, 2)), ('ctc2', (7, 6, 0, 4, 2, 5, 1, 3)), ('3way', (7, 2, 4, 5, 1, 6, 3, 0))]

for a in threebit:
    for b in threebit:
        for c in threebit:
            for d in threebit:
                for alpha in L:
                    S1 = SBox(a[1])
                    S2 = SBox(b[1])
                    S3 = SBox(c[1])
                    S4 = SBox(d[1])
                    Sresult = double_butterfly(S1, S2, S3, S4, alpha, alpha)
                    if Sresult.differential_uniformity() <= 6:
                        print(a[0], b[0], c[0], d[0], alpha)
                        
# Double butterfly with power components and different alpha - 3bits

F.<a> = GF(2**3, name = 'a', modulus = "primitive")
L = F.list()
input = [L[0], L[7], L[1], L[3], L[2], L[6], L[4], L[5]]

power = [z**3 for z in input]
S = SBox(power)
S_inv = S.inverse()
for alpha in L:
    for alpha2 in L:
        Sresult = double_butterfly(S, S, S, S, alpha, alpha2)
        print(alpha, alpha2, Sresult.differential_uniformity())

# Double butterfly with different APN components and alpha - 3bits
        
F.<a> = GF(2**3, name = 'a', modulus = "primitive")
L = F.list()
input = [L[0], L[7], L[1], L[3], L[2], L[6], L[4], L[5]]

threebit= [('cipher', (0, 1, 3, 6, 7, 4, 5, 2)), ('SEA', (0, 5, 6, 7, 4, 3, 1, 2)), ('ctc2', (7, 6, 0, 4, 2, 5, 1, 3)), ('3way', (7, 2, 4, 5, 1, 6, 3, 0))]
for a in threebit:
    for b in threebit:
        for c in threebit:
            for d in threebit:
                for alpha in L:
                    for alpha2 in L:
                        S1 = SBox(a[1])
                        S2 = SBox(b[1])
                        S3 = SBox(c[1])
                        S4 = SBox(d[1])
                        Sresult = double_butterfly(S1, S2, S3, S4, alpha, alpha2)
                        if Sresult.differential_uniformity() <= 6:
                            print(a[0], b[0], c[0], d[0], alpha, alpha2)
                        
# Triple butterfly structure with power components - 3bits
F.<a> = GF(2**3, name = 'a')
L = F.list()
input = [L[0], L[7], L[1], L[3], L[2], L[6], L[4], L[5]]

power = [z**3 for z in input]
S = SBox(power)
S_inv = S.inverse()

for alpha in L:
    Sresult = triple_butterfly(S, S, S, S, S, S, alpha)
    print(alpha, "Differential uniformity: ", Sresult.differential_uniformity(), "Permutation", Sresult.is_permutation())
    
# Triple butterfly structure using APN components - 3-bit
F.<a> = GF(2**3, name = 'a')
L = F.list()
input = [L[0], L[7], L[1], L[3], L[2], L[6], L[4], L[5]]

threebit= [('cipher', (0, 1, 3, 6, 7, 4, 5, 2)), ('SEA', (0, 5, 6, 7, 4, 3, 1, 2)), ('ctc2', (7, 6, 0, 4, 2, 5, 1, 3)), ('3way', (7, 2, 4, 5, 1, 6, 3, 0))]

for a in threebit:
    for b in threebit:
        for c in threebit:
            for d in threebit:
                for e in threebit:
                    for f in threebit:
                        for alpha in L:
                            S1 = SBox(a[1])
                            S2 = SBox(b[1])
                            S3 = SBox(c[1])
                            S4 = SBox(d[1])
                            S5 = SBox(e[1])
                            S6 = SBox(f[1])
                            Sresult = triple_butterfly(S1, S2, S3, S4, S5, S6, alpha)
                            if Sresult.differential_uniformity() <= 6:
                                print(a[0], b[0], c[0], d[0], e[0], f[0], alpha, "Differential uniformity: ", Sresult.differential_uniformity(), "Permutation", Sresult.is_permutation())
                                
# Butterfly replication on 4-bits
def butterfly_four(S1, S2, alpha):
    S1_inv = SBox.inverse(S1)
    results = []
    for xlMain in input:
        ls = []
        for xrMain in input:
            xl = xlMain
            xr = xrMain
            xl = xl + S1[xr] # phase 1
            xl = S1_inv[xl] + (alpha * xr) # phase 2
            temp = xl # swap
            xl = xr
            xr = temp
            xl = xl + (alpha * xr) # phase 3
            xl = S2[xl] + S2[xr] # phase 4
            xlStr = "{0:b}".format(xl.integer_representation()).zfill(4)
            xrStr = "{0:b}".format(xr.integer_representation()).zfill(4)
            ls.append(xlStr + xrStr)
        results.append(ls)

    lsresult = []
    for k in range(0,16):
        for j in range(0,16):
            output = int(results[k][j], 2)
            lsresult.append(output)
    return SBox(lsresult)

# Replacing butterfly components with 4-bit inverse - 4-bit
F.<a> = GF(2**4, name = 'a')
L = F.list()
input = [L[0], L[15], L[1], L[4], L[2], L[8], L[5], L[10], L[3], L[14], L[9], L[7], L[6], L[13], L[11], L[12]]

power = [z**14 for z in input]
S = SBox(power)
S_inv = S.inverse()

for alpha in L:
    Sresult = butterfly_four(S, S, alpha)
    print(alpha, "Differential uniformity: ", Sresult.differential_uniformity(), "Permutation", Sresult.is_permutation())

# Replcaing butterfly components with optimal 4-bit permutations
F.<a> = GF(2**4, name = 'a')
L = F.list()
input = [L[0], L[15], L[1], L[4], L[2], L[8], L[5], L[10], L[3], L[14], L[9], L[7], L[6], L[13], L[11], L[12]]

ls = []
for s in sboxes.items():
    if "Optimal" in s[0]:
        ls.append(s)

for a in ls:
    for b in ls:
        for alpha in L:
            S1 = SBox(list(a[1]))
            S2 = SBox(list(b[1]))
            Sresult = butterfly_four(S1, S2, alpha)
            if Sresult.differential_uniformity() < 12:
                print(a[0], b[0], alpha, Sresult.differential_uniformity(), Sresult.is_permutation())

